In [17]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import os
import math

In [18]:
base_path = '../backend/Data/CSV_Reference/final/'
files = ['banks_final.csv', 'education_final.csv', 'health_final.csv', 'other_final.csv', 'temples_final.csv']
dfs = []
for file in files:
    df = pd.read_csv(os.path.join(base_path, file))
    dfs.append(df)
cafe_df = pd.read_csv(os.path.join(base_path, 'cafe_final.csv'))

In [19]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

combined_df = pd.concat(dfs, ignore_index=True)
combined_df = combined_df.dropna(subset=['lat', 'lng'])
cafe_coords = cafe_df[['lat', 'lng']].dropna().values.tolist()

filtered_pois = []
for idx, poi in combined_df.iterrows():
    has_nearby_cafe = False
    for cafe_lat, cafe_lng in cafe_coords:
        dist = haversine(poi['lat'], poi['lng'], cafe_lat, cafe_lng)
        if dist <= 500:
            has_nearby_cafe = True
            break
    if not has_nearby_cafe:
        filtered_pois.append(poi)

filtered_df = pd.DataFrame(filtered_pois)
poi_data = filtered_df[['lat', 'lng']].values.tolist()

In [20]:
from IPython.display import display

if not filtered_df.empty:
    m = folium.Map(location=[filtered_df['lat'].mean(), filtered_df['lng'].mean()], zoom_start=12)
    HeatMap(poi_data, radius=10, blur=5, min_opacity=0.5).add_to(m)
    folium.LatLngPopup().add_to(m)
    display(m)
else:
    print("No POIs found without cafes within 500m")